# modAL + pyhard- Comparando estratégias

- modAL

    - Amostra por incerteza
    - Amostragem aleatória
    - Consulta por comitê
    - Aprendizado passivo
    - Redução do erro esperado

- Pyhard
    - Harmfulness
    - Usefulness
    - H+U
    - Local Set Cardinality 
    - N2
    - F3

In [1]:
%run -i set_environment

## Bibliotecas

In [2]:
%run -i importing_libraries

## Classificadores

### Algoritmos

In [3]:
%run -i classifiers

### Conjunto de dados

In [4]:
%run -i importing_datasets

## Estatratégias

## modAL

In [102]:
def modal_framework(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy):

    sample_size = 0 #contador de amostras utilizadas pela estratégia
    accuracy_history = []
    f1_history = []
    auc_history = []
    start = timer()    
    
    print(strategy[1])
    if (strategy[1] == "query_by_committee"):
        learner_list = []
        for j in range(1, cost+1): # Loop para criação do comitê

            X_train, X_pool, y_train, y_pool = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
            sample_size = sample_size + len(X_train)
            
            # initializing learner
            learner = ActiveLearner(
                estimator= which_classifier(classifier),
                X_training = X_train, y_training = y_train 
            )
            learner_list.append(learner)
                
#             accuracy_history.append(learner.score(X_pool, y_pool))
#             f1_history.append(compute_f1(learner, X_pool, y_pool, "weighted"))
#             auc_history.append(compute_auc(learner, X_pool, y_pool, "weighted","ovo"))
            
        # assembling the committee
        committee = Committee(
            learner_list=learner_list,
            query_strategy=vote_entropy_sampling)
        
#         sample_size, accuracy_history, f1_history, auc_history = eval(strategy[1] + "(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_pool, y_train, y_pool, learner)")
    
    else: 
        X_train, X_pool, y_train, y_pool = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])

        sample_size = sample_size + len(X_train)
        committee = None

        if (strategy[1] != "random_sampling"):
            learner = ActiveLearner (
                estimator= which_classifier(classifier), #cls,
                query_strategy=uncertainty_sampling,
                X_training = X_train, y_training = y_train # AL AJUSTA O CLASSIFIER 
            )
        else:
            learner = None

#         sample_size, accuracy_history, f1_history, auc_history = eval(strategy[1] + "(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_pool, y_train, y_pool, learner)")
        
            # COLOCAR OU NÃO O PRIMEIRO SCORE?
            # accuracy_history.append(committee.score(X_pool, y_pool))
            # f1_history.append(compute_f1(committee, X_pool, y_pool, "weighted"))
            # auc_history.append(compute_auc(committee, X_pool, y_pool, "weighted", "ovo"))

     
    if (strategy[1] != "random_sampling" and strategy[1] != "query_by_committee"):
        accuracy_history.append(learner.score(X_pool, y_pool))
        f1_history.append(compute_f1(learner, X_pool, y_pool, "weighted"))
        auc_history.append(compute_auc(learner, X_pool, y_pool, "weighted","ovo"))
    
    sample_size, accuracy_history, f1_history, auc_history = eval(strategy[1] + "(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_pool, y_train, y_pool, learner, committee)")
    
    
    end = timer()
    time_elapsed = end - start

    return { "accuracy_history": accuracy_history,
         "f1_history": f1_history,
         "auc_history": auc_history,
         "package": "modAL",
         "time_elapsed": time_elapsed,
         "classifier": classifier,
         "sample_size": sample_size / len(X_raw),
         "Strategy": strategy[0]}

### Amostra por incerteza

In [86]:
def uncertain_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_test, y_train, y_test, learner, committee = None):
    
    total_of_samples = 1
    while (total_of_samples != cost):
      
        idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
        X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][idx]], y_raw[idx_data[idx_bag][TRAIN][idx]]
        
        if classifier_uncertainty(learner, X_train[0].reshape(1,-1)) > 0.2:
            #print("IF", learner.score(X_test, y_test))
            sample_size = sample_size + len(X_train)
            learner.teach(X_train, y_train)
        accuracy_history.append(learner.score(X_test, y_test))
        f1_history.append(compute_f1(learner, X_test, y_test, "weighted"))
        auc_history.append(compute_auc(learner, X_test, y_test, "weighted", "ovo"))
        total_of_samples = total_of_samples + 1
        
    return sample_size, accuracy_history, f1_history, auc_history

### Amostragem aleatória

In [87]:
def random_sampling(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_test, y_train, y_test, learner = None, committee = None):

    for i in range(1, cost+1):

        X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
        sample_size = sample_size + len(X_train)
        
        cls = which_classifier(classifier)
        cls.fit(X_train, y_train)

        accuracy_history.append(cls.score(X_test,y_test))
        f1_history.append(compute_f1(cls, X_test, y_test, "weighted"))
        auc_history.append(compute_auc(cls, X_test, y_test, "weighted", "ovo"))
        
    return sample_size, accuracy_history, f1_history, auc_history

### Consulta por comitê

In [120]:
def query_by_committee(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_pool, y_train, y_pool, learner, committee):    
    
    # query by committee
    for idx in range(cost):
        # print("\t Size of X_pool:", len(X_pool))
        query_idx, query_instance = committee.query(X_pool, n_instances = init_size)
        sample_size = sample_size + len(query_idx)
        
        committee.teach(
            X = X_pool[query_idx],
            y = y_pool[query_idx]
        )

        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx)

        accuracy_history.append(committee.score(X_pool, y_pool))
        f1_history.append(compute_f1(committee, X_pool, y_pool, "weighted"))
        auc_history.append(compute_auc(committee, X_pool, y_pool, "weighted", "ovo"))

    return sample_size, accuracy_history, f1_history, auc_history

### Expected Error Reduction

In [106]:
def exp_error_reduction(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_pool, y_train, y_pool, learner, committee = None):

    total_of_samples = 1
    while (total_of_samples != cost):
        # print("\t Size of X_pool:", len(X_pool))
        exp_error_idx = expected_error_reduction(learner, X_pool, 'binary', n_instances=init_size)

        learner.teach(X_pool[exp_error_idx], y_pool[exp_error_idx])
        sample_size = sample_size + init_size
    
        # X_pool = np.delete(X_pool, exp_error_idx, axis=0)
        # y_pool = np.delete(y_pool, exp_error_idx)
        
        accuracy_history.append(learner.score(X_pool, y_pool))
        f1_history.append(compute_f1(learner, X_pool, y_pool, "weighted"))
        auc_history.append(compute_auc(learner, X_pool, y_pool, "weighted", "ovo"))
        
        total_of_samples = total_of_samples + 1
    
    return sample_size, accuracy_history, f1_history, auc_history

### Expected Model Change

In [107]:
def exp_model_change(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy, sample_size, accuracy_history, f1_history, auc_history, X_train, X_pool, y_train, y_pool, learner, committee = None):

    total_of_samples = 1
    while (total_of_samples != cost):
        # print("\t Size of X_pool:", len(X_pool))
        exp_error_idx = np.random.choice(range(len(X_pool)), size=init_size, replace=False)
        aux = deepcopy(learner)

        aux.teach(X_pool[exp_error_idx], y_pool[exp_error_idx])
        score_aux = aux.score(X_pool, y_pool)
        score_learner = learner.score(X_pool, y_pool)

        if score_aux > score_learner:
            learner = deepcopy(aux)
            sample_size = sample_size + init_size
        
        X_pool = np.delete(X_pool, exp_error_idx, axis=0)
        y_pool = np.delete(y_pool, exp_error_idx, axis=0)
        
        accuracy_history.append(learner.score(X_pool, y_pool))
        f1_history.append(compute_f1(learner, X_pool, y_pool, "weighted"))
        auc_history.append(compute_auc(learner, X_pool, y_pool, "weighted", "ovo"))

        total_of_samples = total_of_samples + 1

    return sample_size, accuracy_history, f1_history, auc_history

## Setup

In [91]:
%run -i setup_functions

## Pyhard Strategies

In [92]:
def pyhard_strategies(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, strategy):
    
    from modAL.uncertainty import classifier_uncertainty
    
    sample_size = 0 #contador de amostras utilizadas pela estratégia
    accuracy_history = []
    f1_history = []
    auc_history = []
    start = timer()
    
    strategy = config(strategy)
    
    # parte randomica inicial da estratégia
    
    X_train, X_test, y_train, y_test = train_test_split(X_raw[idx_data[idx_bag][TRAIN]], y_raw[idx_data[idx_bag][TRAIN]], train_size= len(np.unique(y_raw)) + init_size, stratify = y_raw[idx_data[idx_bag][TRAIN]])
    
    sample_size = sample_size + len(X_train)

    learner = ActiveLearner (
        estimator= which_classifier(classifier), #cls,
        query_strategy=uncertainty_sampling,
        X_training = X_train, y_training = y_train # AL AJUSTA O CLASSIFIER 
    )

    accuracy_history.append(learner.score(X_test, y_test))
    f1_history.append(compute_f1(learner, X_test, y_test, "weighted"))
    auc_history.append(compute_auc(learner, X_test, y_test, "weighted", "ovo"))

    total_of_samples = 1

    #X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size=0.03)

    idx = np.random.choice(range(len(idx_data[idx_bag][TRAIN])), size=init_size, replace=False)
    X_train, y_train = X_raw[idx_data[idx_bag][TRAIN][idx]], y_raw[idx_data[idx_bag][TRAIN][idx]]

    X_rawAndY_raw = np.column_stack([X_raw[idx_data[idx_bag][TRAIN]],y_raw[idx_data[idx_bag][TRAIN]]])
    np.savetxt("data.csv", X_rawAndY_raw, fmt='%i', delimiter=",")
    
    which_pyhard_measure(strategy['measure'])

    !pyhard --no-isa

    df = pd.read_csv('metadata.csv')

    idx = list(df.sort_values(by=strategy['sortby'], ascending=strategy['ascending'])['instances'][:cost])

    X_train = X_raw[idx_data[idx_bag][TRAIN][idx]]
    y_train = y_raw[idx_data[idx_bag][TRAIN][idx]]

    sample_size = cost
    learner.teach(X_train, y_train)
    
    accuracy_history.append(learner.score(X_test, y_test))
    f1_history.append(compute_f1(learner, X_test, y_test, "weighted"))
    auc_history.append(compute_auc(learner, X_test, y_test, "weighted", "ovo"))
    
    end = timer()
    time_elapsed = end - start

    return { "accuracy_history": accuracy_history,
         "f1_history": f1_history,
         "auc_history": auc_history,
         "package": "Pyhard",
         "time_elapsed": time_elapsed,
         "classifier": classifier,
         "sample_size": sample_size / len(X_raw),
         "Strategy": strategy['name']}

In [13]:
datasets = os.listdir('./datasets/luis')
classifiers = ['5NN', 'C4.5', 'NB','RF']


In [14]:
total_performance_history = []

In [15]:
datasets

['1115_teachingAssistant.arff']

In [16]:
metadata = []

for ds in datasets:
    metadata.append(fetch_datasets(ds))

metadata = pd.DataFrame.from_dict(metadata)
metadata

,name,instances,classes,attributes,nominal attributes,majority,minority
0,1115_teachingAssistant,151,3,6,5,34.44,32.45


In [121]:
#modal_strategies = {"Uncertain Sampling":"uncertain_sampling", "Random Sampling":"random_sampling", "Query by Committee":"query_by_committee", "Expected Error Reduction":"exp_error_reduction", "Expected Model Change":"exp_model_change"}
#functions = ["uncertain_sampling", "random_sampling", "query_by_committee", "exp_model_change"]
parameters = "(deepcopy(X_raw), deepcopy(y_raw), idx_data, idx_bag, classifier, init_size, cost)"

modal_strategies = {"Query by Committee":"query_by_committee"}
                   
for ds in tqdm(datasets,  desc ="Dataset"):
    for classifier in classifiers:
        X_raw, y_raw, idx_data, dataset_name = which_arff_dataset(ds)
        #para cada i em idx_bag ("n_splits") (1 a 5)
        for idx_bag in range(n_splits):
            for key, value in modal_strategies.items():
                tqdm.write("Testando: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag+1) + "/" + str(n_splits) + " " + key)
                result = modal_framework(X_raw, y_raw, idx_data, idx_bag, classifier, init_size, cost, [key, value])
                result['dataset'] = ds[:-5]
                total_performance_history.append(result)
                tqdm.write("Passou: " + str(ds[:-5]) + " " + str(classifier) + " " + str(idx_bag+1) + "/" + str(n_splits) + " " + key)

Dataset:   0%|          | 0/1 [00:00<?, ?it/s]

Testando: 1115_teachingAssistant 5NN 1/5 Query by Committee
query_by_committee


ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'

In [67]:
total_performance_history

[{'accuracy_history': [],
  'f1_history': [],
  'auc_history': [],
  'package': 'modAL',
  'time_elapsed': 0.0018398000011075055,
  'classifier': '5NN',
  'sample_size': 0.052980132450331126,
  'Strategy': 'Uncertain Sampling',
  'dataset': '1115_teachingAssistant'},
 {'accuracy_history': [],
  'f1_history': [],
  'auc_history': [],
  'package': 'modAL',
  'time_elapsed': 0.002212100000178907,
  'classifier': '5NN',
  'sample_size': 0.052980132450331126,
  'Strategy': 'Uncertain Sampling',
  'dataset': '1115_teachingAssistant'},
 {'accuracy_history': [],
  'f1_history': [],
  'auc_history': [],
  'package': 'modAL',
  'time_elapsed': 0.0016386999996029772,
  'classifier': '5NN',
  'sample_size': 0.052980132450331126,
  'Strategy': 'Uncertain Sampling',
  'dataset': '1115_teachingAssistant'},
 {'accuracy_history': [],
  'f1_history': [],
  'auc_history': [],
  'package': 'modAL',
  'time_elapsed': 0.002446500000587548,
  'classifier': '5NN',
  'sample_size': 0.052980132450331126,
  'Str

In [ ]:
df

## Visualization

### Preprocessing

In [ ]:
df = pd.DataFrame.from_dict(total_performance_history)

In [ ]:
# compression_opts = dict(method='zip',
#                         archive_name='total_performance_history.csv')  
# df.to_csv('total_performance_history.zip', index=False,
#           compression=compression_opts)  

In [ ]:
# df.read_csv('total_performance_history.csv’, header=None)

### Checking Strategies on Data Frame

In [ ]:
pd.unique(df['Strategy'])

In [ ]:
df = df.explode('accuracy_history')

In [ ]:
df2 = df
df2.groupby(['Strategy', 'classifier']).agg({'performance_history':['mean','std'],'time_elapsed':['mean','std'], 'sample_size':['mean','std']})

In [ ]:
performance_mean = df2.groupby(['Strategy', 'classifier']).mean()
performance_std = df2.groupby(['Strategy', 'classifier']).std()

In [ ]:
performance_mean

### Last loop results (last cost)

In [ ]:
df_last_result = df[:]

In [ ]:
df_last_result['f1_history'] = df_last_result['f1_history'].apply(lambda x: x[-1])
df_last_result['auc_history'] = df_last_result['auc_history'].apply(lambda x: x[-1])
df_last_result['accuracy_history'] = df_last_result['accuracy_history'].apply(lambda x: x[-1])

In [ ]:
df_last_result

In [ ]:
df_last_result_mean = df_last_result.groupby(['Strategy', 'classifier']).mean()
df_last_result_std = df_last_result.groupby(['Strategy', 'classifier']).std()

In [ ]:
df_last_result_mean

### Removing most time expensive strategies to improve visualization 

In [ ]:
df[df.Strategy != "Query by Committee"].sort_values('performance_history', ascending = False)

In [ ]:
df[df.Strategy == "Expected Error Reduction"].sort_values('time_elapsed', ascending = False)

In [ ]:
df.info()

### Plots

In [ ]:
g = sns.relplot(
    data= df,
    x="accuracy_history", y="time_elapsed",
    hue="Strategy", size="sample_size",
    palette=sns.color_palette(n_colors=10), sizes=(100, 300), alpha=0.3
)
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
_ = g.despine(left=True, bottom=True)

In [ ]:
g = sns.relplot(
    data= df_last_result_mean,
    x="accuracy_history", y="time_elapsed",
    hue="Strategy", size="sample_size", style="classifier", height=7,
    palette=sns.color_palette(n_colors=len(pd.unique(df_last_result['Strategy']))), sizes=(100, 300), alpha=0.3
)
g.ax.xaxis.grid(True, "minor", linewidth=.25)
g.ax.yaxis.grid(True, "minor", linewidth=.25)
_ = g.despine(left=True, bottom=True)

In [ ]:
ax = sns.boxplot(data=df_last_result, x="accuracy_history", y="Strategy", orient="h"
                )

In [ ]:
ax = sns.boxplot(data=df_last_result, x="f1_history", y="Strategy", orient="h"
                )

In [ ]:
dfHeatMap = df_last_result.filter(['sample_size','f1_history', 'accuracy_history'], axis=1)
ax = sns.heatmap(dfHeatMap)